In [106]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

## Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.
Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>


In [107]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
KEYWORDS = ['python', 'парсинг']

In [108]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')
# for post in posts:
#     print(post.find_all('a', class_='hub-link'))

In [109]:
for post in posts:
    text = post.find_all('div', class_="post__text post__text-html post__text_v1")
    if text:
        print(text)

[<div class="post__text post__text-html post__text_v1">
      В новом дайджесте делаем модульное приложение из монолита, влюбляемся в градиенты, судимся с Apple, разбираемся с ошибками, создаем погодную станцию и делаем много других полезных вещей. Подключайтесь!<br/>
<br/>
<img src="https://habrastorage.org/webt/vp/oj/db/vpojdbjbocvfrduv-vjcnkzbiyy.png">
</img></div>]
[<div class="post__text post__text-html post__text_v1">
      Недавно я занялся тщательным исследованием VS Code и сделал несколько интересных находок. Как оказалось, в редакторе есть довольно много возможностей и настроек, позволяющих отлично решать те же задачи, которые решают многие популярные расширения.<br/>
<br/>
<a href="https://habr.com/ru/company/ruvds/blog/515300/"><img src="https://habrastorage.org/webt/5p/-i/nl/5p-inliklxbxdgub1i_r4hxado8.png"/></a><br/>
<br/>
Здесь речь пойдёт о шести сферах применения расширений, в которых эти расширения могут быть заменены стандартными механизмами VS Code.<br/>
</div>]
[<d

In [124]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
result = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    texts = post.find_all(['p'])
    # texts = post.find_all('div', class_="post__text post__text-html post__text_v1")

    for text in texts:
            text_lower = text.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([text_lower in desired for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                post_time = re.findall(r'\d{2}:\d{2}',
                                    post.find('span', class_='post__time').text)[0]
                
                result = result.append([[post_time,
                               title_element.text,
                               title_element.attrs.get('href')]],
                             )
result.columns=['дата', 'заголовок', 'ссылка']
result

дата                                          заголовок  \
0  16:03  VKWave — фреймворк для разработки ботов ВКонтакте   
0  15:50           Веб-компоненты в реальном мире (часть 2)   
0  10:39    pyOpenRPA туториал. Управление WEB приложениями   

                             ссылка  
0  https://habr.com/ru/post/515334/  
0  https://habr.com/ru/post/515332/  
0  https://habr.com/ru/post/515310/

In [141]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
result = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    texts = post.find_all(['p'])
    # texts = post.find_all('div', class_="post__text post__text-html post__text_v1")

    for text in texts:
            text_lower = text.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([text_lower in desired for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                post_time = re.findall(r'\d{2}:\d{2}',
                                    post.find('span', class_='post__time').text)[0]
                web = title_element.attrs.get('href')
                req_art = requests.get(web)
                soup_art = BeautifulSoup(req_art.text, 'html.parser')
                post_art= soup_art.find('div',
                    class_="post__text post__text-html post__text_v1").text
       
                result = result.append([[post_time,
                               title_element.text,
                               web,
                               post_art]],
                             )
result.columns=['дата', 'заголовок', 'ссылка', 'текст_статьи']
result

дата                                          заголовок  \
0  16:03  VKWave — фреймворк для разработки ботов ВКонтакте   
0  15:50           Веб-компоненты в реальном мире (часть 2)   
0  10:39    pyOpenRPA туториал. Управление WEB приложениями   

                             ссылка  \
0  https://habr.com/ru/post/515334/   
0  https://habr.com/ru/post/515332/   
0  https://habr.com/ru/post/515310/   

                                        текст_статьи  
0  \nПривет, Хабр!\nСегодня я хочу рассказать о з...  
0  Прошло больше года с моей публикации "Веб-комп...  
0  Долгожданный туториал по управлению сторонними...

In [140]:
result.iloc[0,3]

'\nПривет, Хабр!\nСегодня я хочу рассказать о замечательной библиотеке для разработке ботов ВКонтакте с помощью языка программирования Python.\nVKWave\nVKWave — это фреймворк для разработки ботов ВКонтакте, написанный с помощью asyncio. Основные цели проекта — дать возможность разработчику конфигурировать фреймворк максимально под себя, в тоже время обеспечивая достойную скорость разработки.\nМинимальная требуемая версия Python — 3.7\nВ этой статье я хочу показать вам несколько примеров простых ботов на VKWave, а после вы всегда можете попробовать сами. Фреймворк находится в активной разработке, поэтому документация написанна ещё не до конца, но уже имеется большое количество примеров.\nТакже у нас есть чат в Telegram\nУстановка\nУстановка очень проста и содержит в себе всего одну команду:\npip install vkwave\nА теперь предлагаю перейти к примерам!\nEcho-бот\nСамая простая задача. Бот который отвечает нам тем же самым текстом, который мы ему написали.\n# Импортируем нужные классы.\n# S

## Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.
Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.


In [290]:
EMAIL = ['xxx@x.ru', 'yyy@y.com', 'my@gmail.com']

In [318]:
def hack_check(EMAIL: list = None):
    if EMAIL is None:
        EMAIL = ['xxx@x.ru', 'yyy@y.com', 'my@gmail.com']
    df = pd.DataFrame()
    for mail in EMAIL:
        req = requests.post('https://digibody.avast.com/v1/web/leaks/',
                            json={'email': mail})
        df = df.append(pd.json_normalize(req.json()['value'])[['username',
                                                'leak_date',
                                                'leak_info.source_references',
                                                'leak_info.description',
                                               ]])
    df.leak_date = pd.to_datetime(df.leak_date, unit='ms', origin='unix')
    df.columns=['почта', 'дата утечки', 'источник утечки', 'описание утечки']
    return df
hack_check()

почта дата утечки источник утечки  \
0                xxx@x.ru  2017-12-22              []   
1                xxx@x.ru  2019-02-06              []   
2             aleksey1688  2017-01-31              []   
3    xixixixixixixixixixi  2017-02-14              []   
4                xxx@x.ru  2017-03-08              []   
..                    ...         ...             ...   
101          my@gmail.com  2018-01-20              []   
102          my@gmail.com  2018-09-20              []   
103          my@gmail.com  2018-12-07              []   
104          my@gmail.com  2018-09-10              []   
105          my@gmail.com  2020-08-13              []   

                                       описание утечки  
0    The proliferation of stolen or leaked database...  
1    On January 7, 2019, an online user named Sanix...  
2    In March 2016, CDProjektRed.com.com's forum da...  
3    In July and August 2016, two criminals execute...  
4    This source has been marked as sensitive due t...  
..                                                 ...  
101  At an unconfirmed date, GiaTri.com's database ...  
102  In November 2016, CashCrate's database was all...  
103  In August 2018, the fashion retail site Hautel...  
104  This source has been marked as sensitive due t...  
105  In August 2020, this combolist was shared on a...  

[143 rows x 4 columns]

In [186]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'
TOKEN = '7'
VERSION = '5.103'
SLEEP = 0.33
GROUP = 'netology'
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'count': 50,
    'domain':GROUP,
}
res = requests.get(NEWSFEED_REQUEST, params)
df = pd.DataFrame(res.json()['response']['items'])[['date', 'text']]
df.date = pd.to_datetime(df.date, unit='s')
df

date                                               text
0  2020-08-14 11:00:08  Всем привет! Сегодня первый день карьерного кв...
1  2020-08-16 12:28:00  Подборка книг, которые помогут развить математ...
2  2020-08-16 08:30:24  Продвижение курса — дело непростое. Но у вас к...
3  2020-08-15 11:42:00  Личный бренд работает по принципу зачётки: сна...
4  2020-08-15 08:30:17  Как успехи у ваших студентов? Поздравляем, вы ...
5  2020-08-15 07:42:00  *партнерский пост*\n\nМеждународный конкурс по...
6  2020-08-14 11:00:12  А вот и первое задание карьерного квеста. Мене...
7  2020-08-14 07:42:00  Больше, ещё больше знаний🤓 Разбираемся с дизай...
8  2020-08-13 15:14:00  «Нужно работать так, словно таланта у тебя воо...
9  2020-08-13 07:42:00  Составляем карту пути клиента, открываем калит...
10 2020-08-12 15:42:00  Вечер среды — самое время для поиска формулы б...
11 2020-08-12 07:42:00  Синдром самозванца — склонность преуменьшать с...
12 2020-08-11 14:42:00  Несколько советов, которые помогут держать зад...
13 2020-08-11 07:26:00  Закройте окошко — сквозит 🤧 Откройте отчёт — э...
14 2020-08-10 14:26:00  Фотку котика, загадочный пейзаж или всё-таки с...
15 2020-08-10 07:26:00  Запустили бесплатный курс «Старт в SMM»💛\n\nКа...
16 2020-08-09 14:14:00  Мир 🌏 меняется и нам приходится меняться вмест...
17 2020-08-08 14:14:00  Есть убеждение, что после сорока лет сложно ме...
18 2020-08-08 11:14:00  В этой подборке статей разгадываем загадки Exc...
19 2020-08-07 14:14:00  Остались силы для новый знаний? Свежее расписа...
20 2020-08-07 12:00:08  С места в карьеру: проходите карьерный квест Н...
21 2020-08-07 08:14:00  Могу ли я стать аналитиком без пятерки по школ...
22 2020-08-06 15:14:00  Прочитал страницу, очнулся, а о чем читал и не...
23 2020-08-06 07:42:00  Найти вдохновение не так просто: музу не дожде...
24 2020-08-05 14:42:00  Наш статус: в активном поиске 🔍\n\nДрузья, мы ...
25 2020-08-05 07:42:00  10 августа стартует бесплатный курс по основам...
26 2020-08-04 14:42:00  История выпускника курса «Frontend-разработчик...
27 2020-08-04 07:20:00  Многоуважаемый сударь, прошу прощения покорней...
28 2020-08-03 14:20:00  «Вау, какой цвет стен», «Хочу такую же гардеро...
29 2020-08-03 07:42:00  Как начинающему интернет-магазину продвигаться...
30 2020-08-02 12:42:00  Подборка книг, которые помогут размять креатив...
31 2020-08-01 11:42:00  В новой подборке полезные статьи, которые помо...
32 2020-08-01 07:42:00  *партнёрский пост*\n\nСтартовал региональный э...
33 2020-07-31 15:42:00  Как построить хорошие отношения с коллегами? В...
34 2020-07-31 07:51:00  Лето выходит на финишную прямую, а у нас всё т...
35 2020-07-30 15:42:00  Даже любимое дело может привести к выгоранию, ...
36 2020-07-30 07:14:00  Не стоит прогибаться под изменчивый мир💪🏻 А мо...
37 2020-07-29 15:12:00  Проект или продукт без менеджера — это дрова б...
38 2020-07-29 08:04:03  Друзья, предлагаем заглянуть в будущее 🔮\nИспе...
39 2020-07-28 15:12:00  Что вас вдохновляет? Запах моря, блики солнца,...
40 2020-07-28 07:35:00  Иногда кажется, что в голове куча идей, но сто...
41 2020-07-27 15:14:00  Как зародилась жизнь на Земле? Одни ли мы во В...
42 2020-07-27 07:41:32  Мы все уже в диджитал: общаемся в социальных с...
43 2020-07-26 14:14:00  Как зажигать людей своими идеями?\n⠀\nКрис Анд...
44 2020-07-25 14:14:00  Нетология стала партнером PROBA Awards – между...
45 2020-07-25 11:14:00  Пиши всегда, пиши везде, пиши на суше и в воде...
46 2020-07-24 15:14:00  Женщина — хозяюшка и хорошая мать, которая все...
47 2020-07-24 11:15:00  Верим, что лучший инструмент тайм-менеджмента ...
48 2020-07-24 08:15:00  Второй месяц лета подходит к концу, а мы продо...
49 2020-07-23 15:35:00  Если каждое утро приходится заставлять себя ид...